In [1]:
import os.path
import pickle
import pandas as pd
import numpy as np
import re
import torch 
import nltk

from transformers import AutoTokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline, AutoModelForSeq2SeqLM, T5TokenizerFast, T5ForConditionalGeneration, DataCollatorForSeq2Seq
import datasets
import evaluate
import accelerate

#import Evaluation_Code.Parent as parent ## code for PARENT metric
import Evaluation_Code.Bartscore as bartscore ## code for Bartscore

import json
from ast import literal_eval


e:\Anaconda\envs\CudaSupEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
torch.cuda.empty_cache() 

In [3]:
def ensure_cuda_compatability():
    print(f'Torch version: {torch.__version__}')
    print(f'Cuda version: {torch.version.cuda}')
    print(f'Cudnn version: {torch.backends.cudnn.version()}')
    print(f'Is cuda available: {torch.cuda.is_available()}')
    print(f'Number of cuda devices: {torch.cuda.device_count()}')
    print(f'Current default device: {torch.cuda.current_device()}')
    print(f'First cuda device: {torch.cuda.device(0)}')
    print(f'Name of the first cuda device: {torch.cuda.get_device_name(0)}\n\n')
    torch.backends.cuda.matmul.allow_tf32 = True

def preprocess_model(model_name):
    """
    Setup the model and tokenizer for preprocessing. This will be a pre-trained model collected from huggingface
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name) #, device_map="auto", load_in_8bit=True, load_in_8bit_threshold=4.0) --> tried to bit to bytes
    #tokenizer = T5TokenizerFast.from_pretrained(model_name, is_fast=True)
    #model = T5ForConditionalGeneration.from_pretrained(model_name) #, device_map="auto", load_in_8bit=True, load_in_8bit_threshold=4.0) --> tried to bit to bytes
    #tokenizer = AutoTokenizer.from_pretrained("yhavinga/t5-v1.1-base-dutch-cased")

    #model = AutoModelForSeq2SeqLM.from_pretrained("yhavinga/t5-v1.1-base-dutch-cased")
    model.cuda()

    print('LOGGING: preprocess_model DONE \n')
    return model, tokenizer


def load_CACAPO_data():
    """
    This function retrieves the csv files and creates a dataset
    """
    print('LOGGING: load_CACAPO_data DONE \n')

    return datasets.load_dataset("../Data/Cleaned_data/", data_files={"train": "Train.csv", "dev": "Dev.csv", "test": "Test.csv"})


def preprocess_data(data):
    """
    Tokenize the data
    """
    max_length = 256
    RDFs = data["input"]
    texts = data["output"]

    ## When converting a pandas df to csv (used for loading dataset), a list of lists can transform to a long string
    ## Here we convert it back with literal_eval

    for rdf_iteration, rdf in enumerate(RDFs):
        RDFs[rdf_iteration] = literal_eval(rdf)

    model_inputs = tokenizer(RDFs, truncation=True, padding='max_length', return_tensors='pt',  max_length=max_length, is_split_into_words=True)#.to('cuda')
    
    # specially for seq2seq tokenizer, "Temporarily sets the tokenizer for encoding the targets. Useful for tokenizer associated to sequence-to-sequence models that need a slightly different processing for the labels."
    with tokenizer.as_target_tokenizer():
        target_texts = tokenizer(texts, padding='max_length', truncation=True, return_tensors='pt',  max_length=max_length).input_ids
    
    # target_texts = tokenizer(texts, padding='max_length', truncation=True, return_tensors='pt',  max_length=max_length).input_ids    
    
    model_inputs["labels"] = target_texts#.to('cuda')

    return model_inputs



def transform_datasets(dataset):
    """
    After loading in and creating the initial dataset, the text data is transformed, by tokenizing the input and output texts. The initial dataset is also split into train,val,test for training use.
    NOTE That the test set will not be preprocessed here yet, this will be done in a different function
    """

    ## Create smaller versions of the dataset
    # small_train = dataset["train"].shard(num_shards = 64, index = 0)
    # small_val = dataset["dev"].shard(num_shards = 64, index = 0)
    # small_test = dataset["test"].shard(num_shards = 64, index = 0)

    small_train = dataset["train"]
    small_val = dataset["dev"]
    small_test = dataset["test"]

    # to use the actual articles for evaluation
    true_articles_test = small_test['output']
    # The Parent Metric requires the original RDFs
    test_rdf_input = small_test['input']


    ## Process the data in batches
    small_train = small_train.map(preprocess_data, batched=True, remove_columns=dataset["train"].column_names)
    small_val = small_val.map(preprocess_data, batched=True, remove_columns=dataset["dev"].column_names)
    small_test = small_test.map(preprocess_data, batched=True, remove_columns=dataset["test"].column_names)

    # transform the datasets into torch sensors, as the model will expect this format
    small_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels']) #, device="cuda")
    small_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels']) #, device="cuda")
    small_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels']) #, device="cuda")

    print('LOGGING: transform_datasets DONE \n')

    return small_train, small_val, small_test, true_articles_test, test_rdf_input



def load_eval_metrics():
    """
    Loads in all metrics that will be used later on during evaluation. This is seperated to not load in the metrics a dozen of times during training.
    """
    bleu = datasets.load_metric("bleu")
    rouge = evaluate.load('rouge')
    meteor = evaluate.load('meteor')
    perplexity = evaluate.load("perplexity", module_type="metric")
    bertscore = evaluate.load("bertscore")
    bart_scorer = bartscore.BARTScorer(device='cuda:0', checkpoint='facebook/bart-base') #facebook/bart-base   facebook/bart-large-cnn

    print('LOGGING: load_eval_metrics DONE \n')

    return bleu, rouge, meteor, perplexity, bertscore, bart_scorer
    #return bleu, rouge, meteor, bertscore, bart_scorer



def postprocess_text(preds, labels):
    """
    Supplementary Method called in decode_text.

    Returns list of split decoded labels and predictions for evaluation
    """
    preds = [pred.split() for pred in preds]
    labels = [[label.split()] for label in labels]
    return preds, labels



def decode_text(predictions, labels):
    """
    Supplementary Method called in compute_metrics.

    Returns decoded labels and predictions for evaluation
    """
    if isinstance(predictions, tuple):
            predictions = predictions[0]

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)#.to(device)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)#.to(device)

    return decoded_preds, decoded_labels


def evaluate_texts(decoded_preds, decoded_labels):
    """
    Calculates metrics given a list of decoded predictions and decoded labels
    """
    #post_process for BLEU
    blue_preds, blue_labels = postprocess_text(decoded_preds,  decoded_labels)

    # setup metrics for use
    bleu, rouge, meteor, perplexity, bertscore, bart_scorer = load_eval_metrics()
    #bleu, rouge, meteor, bertscore, bart_scorer = load_eval_metrics()

    # Calculate the metrics
    print(f'\n LOGGING: Calculating Blue')
    bleu_output = bleu.compute(predictions=blue_preds, references=blue_labels)
    print(f'\n LOGGING: Calculating Rouge')
    rouge_output = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    print(f'\n LOGGING: Calculating Meteor')
    meteor_output = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    print(f'\n LOGGING: Calculating Perplexity')
    perp_output = perplexity.compute(predictions=decoded_preds, model_id='gpt2')
    print(f'\n LOGGING: Calculating Bertscore')
    bertscore_output = bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")
    print(f'\n LOGGING: Calculating Bartscore')
    bart_scores_output = bart_scorer.score(srcs=decoded_preds, tgts=decoded_labels, batch_size=4)

    return bleu_output, rouge_output, meteor_output, perp_output, bertscore_output, bart_scores_output



def compute_metrics(pred):
    """"
    Metrics to be evaluated during training and validation
    Metrics used: BLEU, ROUGE, METEOR, Bertscore, BARTScore
    """
    # decode the predictions and labels for eval
    predictions, labels = pred
    decoded_preds, decoded_labels = decode_text(predictions, labels)

    bleu_output, rouge_output, meteor_output, perp_output, bertscore_output, bart_scores_output = evaluate_texts(decoded_preds, decoded_labels)

    ## Huggingsface trainer requires a dict if multiple metrics are used
    return {"blue_output": bleu_output, "rouge_output": rouge_output, "meteor_results": meteor_output,
             "perp_output": perp_output, "bertscore_output": bertscore_output, "bart_scores_output": bart_scores_output}



def set_training_args(model_name, learning_rate, num_train_epochs, evaluation_strategy, generation_num_beams, generation_max_length,
                         gradient_accumulation_steps, per_device_train_batch_size, per_device_eval_batch_size):
    """
    Setup the training arguments that will be used during training.
    """
    #model_name = "t5-fp16-test"
    model_dir = f"../Results/{model_name}"

    training_args = Seq2SeqTrainingArguments(
                output_dir=model_dir,
                learning_rate=learning_rate,
                do_eval=True, # will be set to true if evaluation strategy is set
                do_predict=True, #Whether to run predictions on the test set or not.
                num_train_epochs=num_train_epochs,
                evaluation_strategy= evaluation_strategy,
                eval_steps= 1000, # Number of update steps between two evaluations if evaluation_strategy="steps". Will default to the same value as logging_steps if not set.
                save_steps=500, # Number of updates steps before two checkpoint saves if save_strategy="steps".
                #max_steps=10, # the total number of training steps to perform
                save_total_limit= 10, # the maximum number of models to keep before deleting the oldest one
                predict_with_generate=True, # Whether to use generate to calculate generative metrics (ROUGE, BLEU).
                generation_num_beams=generation_num_beams,  #The num_beams to use on each evaluation loop when predict_with_generate=True. Will default to the num_beams value of the model configuration
                gradient_checkpointing=True, #
                generation_max_length=generation_max_length,
                gradient_accumulation_steps=gradient_accumulation_steps, #Number of updates steps to accumulate the gradients for, before performing a backward/update pass
                per_device_train_batch_size=per_device_train_batch_size, #The batch size per GPU/TPU core/CPU for training.
                per_device_eval_batch_size=per_device_eval_batch_size, #The batch size per GPU/TPU core/CPU for evaluation.
                optim="adafactor", #The optimizer to use: adamw_hf, adamw_torch, adamw_apex_fused, or adafactor.
                #report_to="tensorboard",
                fp16=True,
                #bf16=True, ## should now be possible with rtx 3070
                #tf32=True, #--> moet dan ook torch.backends.cuda.matmul.allow_tf32 = True
                #auto_find_batch_size = True, #Whether to find a batch size that will fit into memory automatically through exponential decay, avoiding CUDA Out-of-Memory errors.
                eval_accumulation_steps=2  #Number of predictions steps to accumulate the output tensors for, 
                                            # before moving the results to the CPU. If left unset, the whole predictions are accumulated on GPU/TPU 
                                            # before being moved to the CPU (faster but requires more memory).
    )

    print('LOGGING: set_training_args DONE \n')

    return training_args


def get_clean_model(model_name):
    """
    Simple function to ensure that a new model is used for finetuning
    """
    return AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda()
    #return T5ForConditionalGeneration.from_pretrained(model_name).cuda() #T5ForConditionalGeneration

#def get_data_Collator(tokenizer):
#    return DataCollatorForSeq2Seq(tokenizer=tokenizer,  return_tensors="pt")

def set_trainer(model_name, training_args, train_ds, val_ds, tokenizer):
    """
    Initializes a trainer
    Takes in: Model name, training arguments, training dataset, validation dataset, and tokenizer
    Returns: Trainer instance
    """
    #collator = get_data_Collator(tokenizer)
    clean_model = get_clean_model(model_name)
    trainer = Seq2SeqTrainer(
                model=clean_model,
                args=training_args,
                train_dataset=train_ds,
                eval_dataset=val_ds,
                compute_metrics=compute_metrics,
                #data_collator = collator,
                tokenizer=tokenizer
                )

    print('LOGGING: set_trainer DONE \n')

    return trainer

def train_and_save(trainer, path_model_name):
    trainer.train()
    trainer.save_model(f"../Models/{path_model_name}")

    print('LOGGING: train_and_save DONE \n')


def get_saved_model(path_model_name):
    saved_model = T5ForConditionalGeneration.from_pretrained(f'../Models/{path_model_name}', local_files_only=True)
    saved_model.cuda()
    tokenizer = AutoTokenizer.from_pretrained(f'../Models/{path_model_name}' ,local_files_only=True)
    return saved_model, tokenizer



def generate_predictions(saved_model, test_set):
    """
    Generates predictions based on the test set, returns a list of predictions and the corresponding "true" articles
    """
    encoded_inputs = test_set.remove_columns("labels")

    # set-up a dataloader to load in the tokenized test dataset
    test_dataloader = torch.utils.data.DataLoader(encoded_inputs,  batch_size=8) #pin_memory=True, --> levert error op: RuntimeError: cannot pin 'torch.cuda.LongTensor' only dense CPU tensors can be pinned

    # generate text for each batch
    all_predictions = []
    for i,batch in enumerate(test_dataloader):
        predictions = saved_model.generate(**batch, max_new_tokens = 100, do_sample=True, num_beams = 5, top_p=0.7, repetition_penalty = 1.3) # .to(device) --> RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)
        all_predictions.append(predictions)

    # flatten predictions
    all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

    print('LOGGING: generate_predictions DONE \n')


    return all_predictions_flattened #, true_articles


def decode_predictions(predictions, tokenizer):
    """
    Decode the predictions made by the model
    """
    decoded_predictions = []

    for iteration, prediction in enumerate(predictions):
        decoded_predictions.append(tokenizer.decode(prediction,skip_special_tokens=True))

    print('LOGGING: decode_predictions DONE \n')

    return decoded_predictions



def evaluate_test_set(path_model_name, test_set, true_articles_test, test_rdf_input):
    """
    Transforms test set, retrieves predictions, and evaluates these predictions
    """
    saved_model, saved_tokenizer = get_saved_model(path_model_name)

    #predictions, test_articles = generate_predictions(saved_model, test_set)
    predictions = generate_predictions(saved_model, test_set)

    decoded_test_predictions = decode_predictions(predictions, saved_tokenizer)

    bleu_output, rouge_output, meteor_output, perp_output, bertscore_output, bart_scores_output = evaluate_texts(decoded_test_predictions, true_articles_test)
    #bleu_output, rouge_output, meteor_output, bertscore_output, bart_scores_output = evaluate_texts(decoded_test_predictions, true_articles_test)


    evaluation_results = {"blue_output": bleu_output, "rouge_output": rouge_output, "meteor_results": meteor_output, "perp_output": perp_output, "bertscore_output": bertscore_output, "bart_scores_output": bart_scores_output}
    #evaluation_results = {"blue_output": bleu_output, "rouge_output": rouge_output, "meteor_results": meteor_output,  "bertscore_output": bertscore_output, "bart_scores_output": bart_scores_output}

    log_results(path_model_name, evaluation_results)

    ##Additional PARENT evaluation
    tables = test_rdf_input
    references = true_articles_test
    generations = decoded_test_predictions
    parent_attempt(path_model_name, generations, references, tables)
    
    ## Huggingsface trainer requires a dict if multiple metrics are used
    return evaluation_results


def write_to_text_parent(path_model_name, decoded_predictions, true_articles, rdfs):
    """
    Parent metric requires text files to work
    """

    with open(f'../Parent_test/{path_model_name}_true_articles.txt', 'w', encoding='utf-8') as f:
        for articles in true_articles:
            f.write(f'{articles} \n')

    with open(f'../Parent_test/{path_model_name}_decode_predictions.txt', 'w', encoding='utf-8') as f:
        for predictions in decoded_predictions:
            f.write(f'{predictions} \n')

    with open(f'../Parent_test/{path_model_name}_rdfs.txt', 'w', encoding='utf-8') as f:
        for pairs in rdfs:
            f.write(f'{pairs} \n')


def prepare_inputs_parent(RDFs):
    """
    Cleans the RDF pairs and transforms them in the proper format so that the parent module can calculate with it.
    """

    attribute_value_pairs = []

    for iteration, inputRDF in enumerate(RDFs):
        split_RDF = inputRDF.split(", ")
        entry=[]
        for connected_pair in split_RDF:
            if '[' in connected_pair:
                connected_pair = connected_pair.replace('[', '')
            if ']' in connected_pair:
                connected_pair = connected_pair.replace(']', '')
            if '_' in connected_pair:
                connected_pair = connected_pair.replace('_', ' ')
            split_pair = tuple(connected_pair.split(' | '))
            entry.append((split_pair))
        attribute_value_pairs.append(entry)
    return attribute_value_pairs


def parent_attempt(path_model_name, generations, references, rdfs):
    """
    The Parent metric needs special treatment, as it only accepts specific inputs and file types.
    """
    prepared_rdfs = prepare_inputs_parent(rdfs)
    write_to_text_parent(path_model_name, generations, references, prepared_rdfs)

    %run -i "~E:/ArriaThesis/MscThesis/Code/Evaluation_Code/Parent.py" --references f"E:/ArriaThesis/MscThesis/Parent_test/{path_model_name}_true_articles.txt" \
                                                                        --generations f"E:/ArriaThesis/MscThesis/Parent_test/{path_model_name}_decode_predictions.txt"  \
                                                                        --tables f"E:/ArriaThesis/MscThesis/Parent_test/{path_model_name}_rdfs.txt"

def log_results(path_model_name, results):
    ## Dutch model has / which ruins the pathing 
    with open(f'../Logging_Results/{path_model_name}_logResults.json', 'w') as convert_file:
        convert_file.write(json.dumps(results))

def fine_tune_model(model_name):
    # ensure cuda compatability
    ensure_cuda_compatability()

    # I instantiate the tokenizer as a global variable, as the .map function in transform_datasets was not working properly. 
    # This should not be an issue, as the tokenizer remains consistent during training and evaluation.
    global tokenizer
    
    #global device
    #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    ## retrieve model and tokenizer from huggingface to prepare dataset
    model, tokenizer = preprocess_model(model_name)
    
    ### retrieve the unprocessed data from the csv files
    entire_dataset = load_CACAPO_data()
    
    # ## process the dataset and split it into its natural train, val, test split
    train_ds, val_ds, test_ds, true_articles_test, test_rdf_inputs = transform_datasets(entire_dataset)

    ### setup the training arguments 
    training_args = set_training_args(model_name=model_name, learning_rate = 0.005, 
                                      num_train_epochs= 2, evaluation_strategy = 'steps', generation_num_beams=5, generation_max_length = 100, 
                                      gradient_accumulation_steps = 2, per_device_train_batch_size= 8, per_device_eval_batch_size= 4)

    ###create a trainer instance 
    trainer = set_trainer(model_name, training_args, train_ds, val_ds, tokenizer)

    # Both mt5 and T5-dutch have / in their name, which makes pathing more chaotic
    if '/' in model_name:
       path_model_name = model_name.replace('/', '_')

    ## Finally fine-tune the model and save it
    train_and_save(trainer, path_model_name)

    testset_evaluation_results = evaluate_test_set( path_model_name, test_ds, true_articles_test, test_rdf_inputs)

    return testset_evaluation_results

def main():
    global model_name
    models = ["yhavinga/t5-v1.1-base-dutch-cased", "google/mt5-small","google/byt5-small"] #  #'google/mt5-base' #'google/mt5-base'      #yhavinga/t5-base-dutch         #t5-base       #google/mt5-small   #google/byt5-base # facebook/nllb-200-distilled-600M

    # for model_type in models:
    #     model_name = model_type
    #     results = fine_tune_model(model_name)
        #print(results)
    model_name = "t5-base"
    results = fine_tune_model(model_name)

In [4]:
main()

Torch version: 1.12.1
Cuda version: 11.3
Cudnn version: 8302
Is cuda available: True
Number of cuda devices: 1
Current default device: 0
First cuda device: <torch.cuda.device object at 0x000002137F039F08>
Name of the first cuda device: NVIDIA GeForce RTX 3070




e:\Anaconda\envs\CudaSupEnv\lib\site-packages\transformers\convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Using custom data configuration Cleaned_data-4b8a9b4c5ecd8560
Found cached dataset csv (C:/Users/Simon/.cache/huggingface/datasets/csv/Cleaned_data-4b8a9b4c5ecd8560/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


LOGGING: preprocess_model DONE 

LOGGING: load_CACAPO_data DONE 



100%|██████████| 3/3 [00:00<00:00, 99.91it/s]
Parameter 'function'=<function preprocess_data at 0x0000021309CEB798> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
  0%|          | 0/16 [00:00<?, ?ba/s]e:\Anaconda\envs\CudaSupEnv\lib\site-packages\transformers\tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate

LOGGING: transform_datasets DONE 

LOGGING: set_training_args DONE 



Using cuda_amp half precision backend
***** Running training *****
  Num examples = 15290
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 1912


LOGGING: set_trainer DONE 



  0%|          | 0/1912 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


RuntimeError: CUDA out of memory. Tried to allocate 978.00 MiB (GPU 0; 8.00 GiB total capacity; 6.36 GiB already allocated; 0 bytes free; 6.38 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Models voor experimenten:


## Works
- yhavinga/t5-base-dutch --> puurly dutch 
- t5-base --> english
- google/mt5-small
- google/byt5-base 

## Doesnt work
- google/mT5-base --> multilingual --> Lijkt te werken op Colab
- google/byt5-base --> Testen op Colab
- - facebook/nllb-200-distilled-600M --> needs further testing --> testen op Colab

## Needs testing




In [1]:
# Als dataloader, bf16 en tf32 niet werken --> check DeepSpeed ZeRO --> https://huggingface.co/docs/transformers/perf_train_gpu_one#choice-of-gpu
#https://github.com/huggingface/transformers/issues/14608

# Let op:

Als je deepseed gebruikt, lees Shared Configuration
https://huggingface.co/docs/transformers/main_classes/deepspeed#deployment-in-notebooks

1. https://huggingface.co/docs/transformers/main_classes/deepspeed
2. https://www.deepspeed.ai/docs/config-json/
3. https://github.com/microsoft/DeepSpeedExamples/blob/master/gan/gan_deepspeed_config.json
4. https://huggingface.co/blog/zero-deepspeed-fairscale
5. https://github.com/huggingface/transformers/issues/8771